В питоне есть т.н. магические методы (закрытые в двойное подчеркивание). Самые известные примеры \_\_init\_\_, \_\_str\_\_,
\_\_repr\_\_ и тд.

Разберем некоторые из них. Первый метод - \_\_new\_\_. Он позволяет создать т.н. называемый конструктор, поскольку является единственным методом, который при инициализации инстанса вызвается раньше, чем \_\_init\_\_. Часто используется, когда мы хотим контролировать кол-во инстансов объекта. Ниже пример с лимитом 3

In [1]:
class Mark:
    _instances=[]
    limit=3 #т.е. мы не хотим иметь больше 5 инстансов
    def __new__(cls,*args,**kwargs):
        print("i am here")
        if len(cls._instances)>=cls.limit:
            print(1)
            raise RuntimeError(f"Не шмогла я, не шмогла, лимит всего {cls.limit}")
        instance = super().__new__(cls)
        cls._instances.append(instance)
        return instance
    
    def __init__(self,name):
        print ("here is init babe")
        self.name = name

lister = ['kurp','burp', 'durp', 'lurp']
res=[]
for i in lister:
    res.append(Mark(i))
print((len(res[1]._instances)))   

i am here
here is init babe
i am here
here is init babe
i am here
here is init babe
i am here
1


RuntimeError: Не шмогла я, не шмогла, лимит всего 3

Следующий метод: \_\_eq_\_\. Он позволяет определить что происходит при сравнении инстансов между собой (иными словами, перегружает оператор "==")

In [2]:
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email
        
    def __eq__(self,obj):
        return self.email == obj.email

jane = User('Jane Doe', 'jdoe@example.com')
joe = User('Joe Doe', 'jdoe@example.com')


print(jane == joe)

True


Метод \_\_hash\_\_ позволяет описать поведение функции hash при применении к нашему объекту. Это особенно полезно,
если мы собираемся писать инстансы в хэшируемый объект, например в словарь. Тогда, если у нас есть два объекта, с разными, например, именами, но одинаковым хэшем, в словаре будет создана только одна запись. На примере будет понятнее:

In [5]:
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email
        
    def __hash__(self):
        return hash(self.email)  #получается, если у нас две записи с одним имейлом, у них все равно будет одинаковый хэш
        
    def __eq__(self,obj):
        return self.email == obj.email

jane = User('Jane Doe', 'jdoe@example.com')
joe = User('Joe Doe', 'jdoe@example.com')

print(hash(jane))
print(hash(joe))
hashable_type = {user:user.name for user in [jane,joe]}
print(hashable_type) #только один объект добавлен в словарь, хотя итерация шла по двум переменным. Очередное напоминание того,
#что в питоне переменные - это просто указатели, и в данном случае они указывают на один объект

-7899787547237131296
-7899787547237131296
{<__main__.User object at 0x0000027C818A9B00>: 'Joe Doe'}


Следующие методы: \_\_getattr\_\_ и \_\_getattribute\_\_. Их часто путают, но у них есть фундаментальное различие - \_\_getattribute\_\_ вызывается каждый раз, когда мы пытаемся получить какой-то аттрибут (т.е. при любой записи формата instance.attribute), а метод \_\_getattr\_\_ вызывается только если атрибут получить не удалось. Тонкий момент - \_\_setattr\_\_ это аналог \_\_getattribute\_\_ для присвоения атрибута, т.е. он вызывается каждый раз, когда мы пытаемся присвоить значение атрибуту.

In [11]:
class Researcher:
    def __getattr__(self, name):
        return 'Nothing found :()\n'
    
    def __getattribute__(self, name):
        print('Looking for {}'.format(name))
        return object.__getattribute__(self, name)  #__getattribute__ обязательно должен обращаться к этому же методу
                                                    #родительского класса, иначе - бесконечная рекурсия, с сетом та же тема
    

obj = Researcher()

print(obj.attr)
print(obj.method)
print(obj.DFG2H3J00KLL)

Looking for attr
Nothing found :()

Looking for method
Nothing found :()

Looking for DFG2H3J00KLL
Nothing found :()



Далее поговорим о методе \_\_call\_\_. Его часто путают с \_\_init\_\_. Однако на деле они достаточно серьезно отличаются. Инит вызывается при __инициализации инстанса__, а \_\_call\_\_ при __вызове инстанса как функции__. Кстати у функций тоже есть метод колл.

In [19]:
class Vasya:
    def __init__(self,name):
        print("init here bro")
        self.name = name
    def __call__(self, *args):
        print("call here bro")
        return sum(list(args))

vas = Vasya('sos')  #тут работает init
print(vas(2,3,4))   #тут работает call


init here bro
call here bro
9


Метод \_\_add\_\_ перегружает оператор +. Для остальных операторов есесна тоже есть свои методы.

\_\_getitem\_\_ и \_\_setitem\_\_ опредляют чтение и присвоение значения по ключу (типа как в словаре, когда ключ в квадратных скобочках). 

Особенного внимания заслуживают методы \_\_iter\_\_ и \_\_next\_\_, которые определяют поведение объекта при итерации. Так, если у объекта не определен \_\_iter\_\_, то по нему невозможно будет итерироваться. Итого, итератор в python — это любой объект, реализующий метод \_\_next\_\_ без аргументов, который должен вернуть следующий элемент или ошибку StopIteration. Также он реализует метод \_\_iter\_\_ и поэтому сам является итерируемым объектом.

In [21]:
class SquareIterator:
    def __init__(self, start, end):
        self.current = start
        self.end = end
        
    def __iter__(self):  #в 99% случаев итератор должен возвращать self, чтобы некст понимал, что делает
        return self
    
    def __next__(self):
        if self.current >= self.end:
            raise StopIteration

        result = self.current ** 2
        self.current += 1
        return result
    
    
for num in SquareIterator(1, 4):
    print(num)

1
4
9


# Контекстные менеджеры

Контекстные менеджеры - это объекты, в которых определены методы _\_\enter\_\_ и \_\_exit\_\_. Самый наглядный пример - __open__. Менеджеры контекста позволяют выполнить какие-то действия для настройки или очистки (например. закрытие файла как в open), если создание объекта завернуто в слово with. 

\_\_enter\_\_ собственно позволяет нам войти в этот самый контекст, в котором будут производиться действия пока блок with не закончится. Важно, что то что возвращает \_\_enter\_\_ и будем тем значением, которое примэппится к значению после улючевого слова __as__.

\_\_exit\_\_, таким образом, определяет действие, которое должно быть выполнно при завершении или прерывании выполнения кода в блоке __with .. as__. Чаще всего это закрытие файла, разрыв соединения и т.п.

In [22]:
class open_file:
    def __init__(self, filename, mode):
        self.f = open(filename, mode)
    
    def __enter__(self):
        return self.f
    
    def __exit__(self, *args):
        self.f.close()

In [23]:
with open_file('test.log', 'w') as f:
    f.write('Inside `open_file` context manager')

with open_file('test.log', 'r') as f:
    print(f.readlines())

['Inside `open_file` context manager']


Есть стандартная библиотека contextlib, в которой много всяких полезных менеджеров контекста.

# Дескрипторы

Дескрипторы - специальные классы, которые позволяют переопределить повведение объектов при доступе, назначении и удалении атрибутов. Это наиболее низкоуровневый механизм, на котором основываются декораторы типа @property, @staticmethod, @classmethod. Иными словами, при использовании этих декораторов мы создаем инстанс соответствующего класса-дескриптора (property, staticmethod или classmethod). 

Чтобы класс считался дескриптором достаточно определить хотя бы один из методов \_\_get\_\_, \_\_set\_\_, \_\_delete\_\_. Если определен только \_\_get\_\_ - перед нами non-data descriptor, если все три, то data descriptor. Чтобы понять, в чем принципиальное отличие между ними, нужно сделать небольшое отступление, объясняющее что собственно делают дескрипторы.


In [1]:
class MyDescriptor():
    """
    A simple demo descriptor
    """
    def __init__(self, initial_value=None, name='my_var'):
        self.var_name = name
        self.value = initial_value
 
    def __get__(self, obj, objtype):
        print('Getting', self.var_name)
        return self.value
 
    def __set__(self, obj, value):
        msg = 'Setting {name} to {value}'
        print(msg.format(name=self.var_name, value=value))
        self.value = value
 
class MyClass():
    desc = MyDescriptor(initial_value='Mike', name='desc')
    normal = 10
    
c = MyClass()
print(c.desc)
print(c.normal)
c.desc = 100
print(c.desc)

Getting desc
Mike
10
Setting desc to 100
Getting desc
100


Как мы видим из примера выше, при вызове вида __obj.attribute__ на самом деле вызывается метод дескриптора \_\_get\_\_. В случае с \_\_set\_\_ ситуация аналогичная. Если вдаватьсяя в технические детали, то при любом вызове obj.attribute вызывается \_\_getattribute\_\_, внутри которого прописана логика поиска, в которой у \_\_get\_\_ самый высокий приоритет. Таким образом, если мы имеем дело с дескриптором, вызов типа obj.attribute (т.е. вызов из __инстанса, а не из класса__) трансформируется в __type(obj).\_\_dict\_\_\['atribute'\].\_\_get\_\_(obj, type(obj))__. При вызове из класса типа __Class.attr__ он превращается в __Class.\_\_dict\_\_\['attr'\].\_\_get\_\_(None, Class)__.

Разницу в вызове чего бы то ни было из класса и инстанса отлично демонстрировать на примере методов/функций. Раньше в питоне различали bound method, undound method и function. Из названия понятно что bound метод привязан к конкретному инстансу, а unbound, который в третьем питоне слился с function, к конкретному инстансу не привязан и может быть вызван как обычная функция, без инициализации инстанса.

In [3]:
class Class:
    def method(self):
        pass    
    
obj = Class()    

print(obj.method)
print(Class.method)


<bound method Class.method of <__main__.Class object at 0x0000020C46CA9588>>
<function Class.method at 0x0000020C46C83730>


Так вот, разница между data и non-data descriptors заключается в том, что у них разная логика поиска (lookup chain). В случае с data descriptor у нас \_\_get\_\_ всегда в приоритете, а вот в случае с non-data - поиск атрибута в \_\_dict\_\_ инстанса будет приоритетен. Пример:

In [10]:
class A(object):
   def __get__(self, obj, objtype):
       print("hello from get A")
   def __set__(self, obj, val):
       print ("hello from set A")
#non data descriptor
class B(object):
   def __get__(self, obj, objtype):
       print ("hello from get B")

class C(object):
   #our data descriptor
   a = A()
   #our non data descriptor
   b = B()

c = C()

In [11]:
c.a

hello from get A


In [12]:
c.b

hello from get B


In [13]:
c.a=0
c.a  #notice after reassignment, A.__get__ is still called

hello from set A
hello from get A


In [14]:
c.b=0   #instance variable with the same name as the non data-descriptor
c.b     #__get__ is not called

0

__NB__. Атрибуты, которые я хочу сделать дескрипторами в своем классе, не нужно инциализировать через \_\_init\_\_, потому что в таком случае я их __замаскирую__ и дескриптор лишится всякого смысла. Пример:

In [16]:
class D(object):

    "The Descriptor"

    def __init__(self, x = 1395):
        self.x = x

    def __get__(self, instance, owner):
        print ("getting", self.x)
        return self.x


class C(object):

    d = D()   #атрибут d стал дескриптором

    def __init__(self, d):      # засунули его в инит
        self.d = d

In [18]:
c = C(4)
c.d   # принт не вывелся, т.е. __get__ не был вызван

4